Exercise 1 Simple neural network
Implement a simple neural network that predicts the class of iris plant basing on its parameters. 
Use 2 hidden layers. The layers should be fully connected with RELu activation function, except for the output layer, which should use softmax function.
 No regularisation or optimalisation are needed. No batching is needed. 


In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()


In [2]:
print(iris.DESCR)


.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [3]:
X = iris.data
y = iris.target

In [4]:
X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.25 , random_state=42,shuffle=True)

In [75]:
class Neuron():
    def __init__(self,in_features,out_features):
        self.weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
        self.bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
        
    def forward(self, X):
        return X @ self.weights + self.bias


    

In [ ]:
def hidden_unit(X , in_features, out_features):
    weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
    bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
    
    return X @ weights + bias

def Relu(X):
    return np.maximum(X,0)

def Softmax(X):
    X_exp = np.exp(X)
    row_sum = np.sum(X_exp,axis=1,keepdims=True)
    return X_exp/row_sum  

In [76]:
N_1 = Neuron(4,10)
N_2 = Neuron(10,10)
N_3 = Neuron(10,4)


In [77]:
X_1 = Relu(N_1.forward(X))   # it has some weight and bias

In [78]:
X_2 = Relu(N_2.forward(X_1)) # it has some weight and bias

In [79]:
X_softmax = Softmax(N_3.forward(X_2)) # it has some weight and bias 

In [80]:
X_softmax

array([[0.19601341, 0.38023687, 0.29459736, 0.12915237],
       [0.19119685, 0.36921992, 0.30208864, 0.13749459],
       [0.19822099, 0.36605168, 0.29997255, 0.13575479],
       [0.20074279, 0.35940163, 0.2986001 , 0.14125547],
       [0.2001306 , 0.37792394, 0.29289464, 0.12905082],
       [0.19999462, 0.38911463, 0.28572131, 0.12516945],
       [0.20541225, 0.36314664, 0.29571309, 0.13572803],
       [0.19750343, 0.37482956, 0.29471641, 0.13295061],
       [0.2002296 , 0.35259826, 0.30259168, 0.14458047],
       [0.19413036, 0.36911254, 0.29878061, 0.13797649],
       [0.1940731 , 0.39043042, 0.29051911, 0.12497737],
       [0.20311966, 0.36713381, 0.29303744, 0.1367091 ],
       [0.1933732 , 0.36630197, 0.30149411, 0.13883072],
       [0.20109575, 0.35446046, 0.30455508, 0.13988871],
       [0.18709684, 0.41000974, 0.28957495, 0.11331847],
       [0.1999229 , 0.40587695, 0.28051642, 0.11368373],
       [0.19543738, 0.39431257, 0.29067384, 0.11957621],
       [0.19600678, 0.37999624,